In [1]:
import os
from pprint import pprint

In [2]:
%ls

langchain_0shot_complex/         matching_one_shot_simple.ipynb
langchain_0shot_simple/          matching_zero_shot_complex.ipynb
matched_complex_oneshot.csv      matching_zero_shot_simple.ipynb
matched_complex_zeroshot.csv     schema_0shot_complex/
matched_simple_oneshot.csv       schema_0shot_simple/
matched_simple_zeroshot.csv      schema_1shot_complex/
matching_langchain_simple.ipynb  schema_1shot_simple/
matching_one_shot_complex.ipynb


In [3]:
data_path = '../RMPEA Dataset filtered Aug 30 23.csv'
assert os.path.exists(data_path)

In [4]:
extraction_dir = 'langchain_0shot_simple/'

In [5]:
%pip install pymatgen

Note: you may need to restart the kernel to use updated packages.


In [6]:
# %pip install numpy==1.24.4

# Read paper dataset

In [7]:
import pandas as pd

In [8]:
paper_df = pd.read_csv(data_path)
display(paper_df.shape)
display(paper_df.head(5))

(971, 23)

,IDENTIFIER: Reference ID,FORMULA,PROPERTY: Microstructure,PROPERTY: Processing method,PROPERTY: BCC/FCC/other,PROPERTY: grain size ($\mu$m),PROPERTY: Exp. Density (g/cm$^3$),PROPERTY: Computed Density (g/cm$^3$),PROPERTY: HV,PROPERTY: Type of test,...,PROPERTY: Elongation (%),PROPERTY: Elongation plastic (%),PROPERTY: Exp. Young modulus (GPa),PROPERTY: Computed Young modulus (GPa),PROPERTY: O content (wppm),PROPERTY: N content (wppm),PROPERTY: C content (wppm),REFERENCE: doi,REFERENCE: year,REFERENCE: title
0,7,Al0.025 Cr0.2 Mo0.2 Nb0.2 Ta0.025 Ti0.2 V0.05 ...,BCC+Laves,CAST,other,NaN,7.44,7.5,NaN,C,...,NaN,NaN,NaN,185.0,NaN,NaN,NaN,10.1016/j.jallcom.2020.155700,2020.0,Combinatorial development of the low-density h...
1,7,Al0.025 Cr0.2 Mo0.2 Nb0.2 Ta0.05 Ti0.2 V0.025 ...,BCC+Laves,CAST,other,NaN,7.77,7.8,NaN,C,...,NaN,NaN,NaN,187.0,NaN,NaN,NaN,10.1016/j.jallcom.2020.155700,2020.0,Combinatorial development of the low-density h...
2,7,Al0.025 Cr0.2 Mo0.2 Nb0.2 Ta0.075 Ti0.2 Zr0.1,BCC+Laves,CAST,other,NaN,8.06,8.1,NaN,C,...,NaN,NaN,NaN,188.0,NaN,NaN,NaN,10.1016/j.jallcom.2020.155700,2020.0,Combinatorial development of the low-density h...
3,7,Al0.025 Cr0.2 Mo0.2 Nb0.2 Ti0.2 V0.075 Zr0.1,BCC+Laves,CAST,other,NaN,7.19,7.3,NaN,C,...,NaN,NaN,NaN,184.0,NaN,NaN,NaN,10.1016/j.jallcom.2020.155700,2020.0,Combinatorial development of the low-density h...
4,1,Al0.038 Mo0.192 Nb0.192 Ta0.192 Ti0.192 V0.192,BCC,CAST,BCC,NaN,NaN,9.0,474.0,C,...,16.0,NaN,NaN,169.0,NaN,NaN,NaN,10.1016/j.msea.2020.139275,2020.0,Effects of Al addition on the microstructures ...


## Analyzing unique keys

In [9]:
gb = paper_df.groupby(['REFERENCE: doi','FORMULA','PROPERTY: YS (MPa)'])

In [10]:
len(gb.groups)

883

In [11]:
for group in gb.groups:
  print(gb.indices)
  break

{('10.1002/adem.202100765', 'Mo0.25 Re0.25 Ta0.25 W0.25', 172.0): array([823]), ('10.1002/adem.202100765', 'Mo0.25 Re0.25 Ta0.25 W0.25', 1075.0): array([822]), ('10.1002/adma.201701678', 'Hf0.25 Ta0.25 Ti0.25 Zr0.25', 1356.0): array([622]), ('10.1002/adma.201701678', 'Hf0.278 Ta0.167 Ti0.278 Zr0.278', 750.0): array([627]), ('10.1002/adma.201701678', 'Hf0.286 Ta0.143 Ti0.286 Zr0.286', 687.0): array([628]), ('10.1002/adma.201701678', 'Hf0.294 Ta0.118 Ti0.294 Zr0.294', 354.0): array([631]), ('10.1007/s00339-019-2484-1', 'Hf0.2 Sc0.2 Ti0.2 Y0.2 Zr0.2', 793.0): array([569]), ('10.1007/s00339-019-2484-1', 'Hf0.25 Sc0.25 Ti0.25 Zr0.25', 1001.0): array([621]), ('10.1007/s00339-019-2484-1', 'Hf0.25 Ti0.25 Y0.25 Zr0.25', 554.0): array([625]), ('10.1007/s00339-019-2484-1', 'Hf0.333 Ti0.333 Zr0.333', 773.0): array([639]), ('10.1007/s10853-012-6260-2', 'Hf0.2 Nb0.2 Ta0.2 Ti0.2 Zr0.2', 92.0): array([470]), ('10.1007/s10853-012-6260-2', 'Hf0.2 Nb0.2 Ta0.2 Ti0.2 Zr0.2', 295.0): array([469]), ('10.1007

In [12]:
v_groups = {group:indices for group, indices in gb.indices.items() if len(indices) > 1}
v_groups

{('10.1016/j.actamat.2019.06.006',
  'Al0.25 Nb0.25 Ta0.25 Ti0.25',
  740.0): array([249, 250]),
 ('10.1016/j.actamat.2019.06.006',
  'Cr0.25 Mo0.25 Ta0.25 Ti0.25',
  1162.0): array([330, 331]),
 ('10.1016/j.msea.2017.10.073',
  'Hf0.15 Nb0.2 Ta0.1 Ti0.3 Zr0.25',
  1020.0): array([402, 409]),
 ('10.1016/j.msea.2017.10.073',
  'Hf0.15 Nb0.2 Ta0.1 Ti0.3 Zr0.25',
  1370.0): array([396, 400])}

In [13]:
pd.set_option("display.max_columns", None)
paper_df.iloc[[396, 400]]

,IDENTIFIER: Reference ID,FORMULA,PROPERTY: Microstructure,PROPERTY: Processing method,PROPERTY: BCC/FCC/other,PROPERTY: grain size ($\mu$m),PROPERTY: Exp. Density (g/cm$^3$),PROPERTY: Computed Density (g/cm$^3$),PROPERTY: HV,PROPERTY: Type of test,PROPERTY: Test temperature ($^\circ$C),PROPERTY: YS (MPa),PROPERTY: UTS (MPa),PROPERTY: Elongation (%),PROPERTY: Elongation plastic (%),PROPERTY: Exp. Young modulus (GPa),PROPERTY: Computed Young modulus (GPa),PROPERTY: O content (wppm),PROPERTY: N content (wppm),PROPERTY: C content (wppm),REFERENCE: doi,REFERENCE: year,REFERENCE: title
396,74,Hf0.15 Nb0.2 Ta0.1 Ti0.3 Zr0.25,BCC,WROUGHT,BCC,NaN,NaN,8.4,NaN,C,-153.0,1370.0,NaN,NaN,NaN,NaN,103.0,NaN,NaN,NaN,10.1016/j.msea.2017.10.073,2018.0,Mechanical properties and thermally activated ...
400,74,Hf0.15 Nb0.2 Ta0.1 Ti0.3 Zr0.25,BCC,WROUGHT,BCC,NaN,NaN,8.4,NaN,C,-103.0,1370.0,NaN,NaN,NaN,NaN,103.0,NaN,NaN,NaN,10.1016/j.msea.2017.10.073,2018.0,Mechanical properties and thermally activated ...


# Read extraction jsons

In [14]:
os.listdir(extraction_dir)


['10.1016.j.matlet.2020.127372.jsonl',
 '10.1016.j.mtla.2020.100627.jsonl',
 'failed_DOIs.json',
 '10.1016.j.actamat.2019.06.006.jsonl',
 '10.1016.j.intermet.2019.01.004.jsonl',
 '10.1007.s00339-019-2484-1.jsonl',
 '10.3390.e16020870.jsonl',
 '10.1080.21663831.2016.1221861.jsonl',
 '10.1016.j.ijrmhm.2020.105195.jsonl',
 '10.1016.j.msea.2016.07.102.jsonl',
 '10.1007.s42864-021-00111-8.jsonl',
 '10.1016.j.actamat.2019.06.032.jsonl',
 '10.1016.j.jallcom.2014.11.064.jsonl',
 '10.1016.j.msea.2017.10.073.jsonl',
 '10.1016.j.ijrmhm.2019.105132.jsonl',
 '10.1016.j.matlet.2014.11.162.jsonl',
 '10.1016.j.msea.2021.141908.jsonl',
 '10.1016.j.msea.2012.12.018.jsonl',
 '10.1016.j.msea.2020.139774.jsonl',
 '10.1007.s11837-020-04557-y.jsonl',
 '10.1016.j.actamat.2016.01.018.jsonl',
 '10.1007.s11837-012-0366-5.jsonl',
 '10.1016.j.scriptamat.2021.113919.jsonl',
 '10.1016.j.msea.2017.12.021.jsonl',
 '10.1016.j.scriptamat.2019.07.033.jsonl',
 '10.1016.j.intermet.2015.03.013.jsonl',
 '10.1016.j.matlet.201

In [64]:
import json
extraction_dfs = []
extraction_dois = set()
for extraction_fn in os.listdir(extraction_dir):
  if extraction_fn == 'failed_DOIs.json':
    continue
  print(extraction_fn)
  extraction_path = os.path.join(extraction_dir, extraction_fn)
  with open(extraction_path) as f:
    obj = json.load(f)

  doi = extraction_fn.replace('.jsonl','')
  extraction_dois.add(doi)
  
  try:
    df = pd.DataFrame(obj)
    df['doi'] = doi
    
  except:
    print(extraction_fn)
    print(obj)
    continue
  
  
  extraction_dfs.append(df)
  
  
  
  ''' commenting out everything below this line for time being
  
  # if 'extracted_info' in obj['extracted_info'].keys():
  #   print('a list of dict with key')
  # else:
  #   print(obj['extracted_info'])

  # Slightly redundant schema
  pprint(obj['extracted_info'])
  records = [entry['extracted_info'] if 'extracted_info' in obj['extracted_info'].keys() else entry for entry in obj['extracted_info']]
  pprint(records)
  df = pd.DataFrame(records)
  df['doi'] = doi

  display(df.shape)
  display(df.head(1))
  extraction_dfs.append(df)
  print('\n','*'*50,'\n')
  '''

print('Extraction DFs:',len(extraction_dfs))

10.1016.j.matlet.2020.127372.jsonl
10.1016.j.mtla.2020.100627.jsonl
10.1016.j.actamat.2019.06.006.jsonl
10.1016.j.intermet.2019.01.004.jsonl
10.1007.s00339-019-2484-1.jsonl
10.3390.e16020870.jsonl
10.1080.21663831.2016.1221861.jsonl
10.1016.j.ijrmhm.2020.105195.jsonl
10.1016.j.msea.2016.07.102.jsonl
10.1007.s42864-021-00111-8.jsonl
10.1016.j.actamat.2019.06.032.jsonl
10.1016.j.jallcom.2014.11.064.jsonl
10.1016.j.msea.2017.10.073.jsonl
10.1016.j.ijrmhm.2019.105132.jsonl
10.1016.j.matlet.2014.11.162.jsonl
10.1016.j.msea.2021.141908.jsonl
10.1016.j.msea.2012.12.018.jsonl
10.1016.j.msea.2020.139774.jsonl
10.1007.s11837-020-04557-y.jsonl
10.1016.j.actamat.2016.01.018.jsonl
10.1007.s11837-012-0366-5.jsonl
10.1016.j.scriptamat.2021.113919.jsonl
10.1016.j.msea.2017.12.021.jsonl
10.1016.j.scriptamat.2019.07.033.jsonl
10.1016.j.intermet.2015.03.013.jsonl
10.1016.j.matlet.2016.08.060.jsonl
10.1016.j.vacuum.2020.109614.jsonl
10.1002.adem.200300567.jsonl
10.1016.j.ijrmhm.2020.105467.jsonl
10.1016.j

In [65]:
extraction_dfs

[                              formula  yield_strength  grain_size  experimental_density  hardness  test_temperature  ultimate_tensile_strength  elongation  elongation_plastic                                           evidence                           doi
 0  Al 20 Cr 10 Nb 15 Ti 20 V 25 Zr 10          1535.0         0.0                  5.55       0.0              22.0                        0.0         0.6                 0.0  At 22 °C, the as-cast RHEA fractured in the el...  10.1016.j.matlet.2020.127372
 1  Al 20 Cr 10 Nb 15 Ti 20 V 25 Zr 10          1000.0         0.0                  5.55       0.0             800.0                        0.0         0.0                 0.0  At 800 °C, stress-strain curves of the RHEA in...  10.1016.j.matlet.2020.127372
 2             Al20Cr10Nb15Ti20V25Zr10             0.0         0.0                  0.00       0.0               0.0                        0.0         0.0                 0.0  In summary, the presented data show that the A...  1

In [66]:
colname_map = {'FORMULA':'formula',
               'PROPERTY: grain size ($\mu$m)':'grain_size',
               'PROPERTY: YS (MPa)':'yield_strength',
               'PROPERTY: Exp. Density (g/cm$^3$)':'experimental_density',
               'PROPERTY: HV':'hardness',
               'PROPERTY: Test temperature ($^\circ$C)':'test_temperature',
               'PROPERTY: UTS (MPa)':'ultimate_tensile_strength',
               'PROPERTY: Elongation (%)':'elongation',
               'PROPERTY: Elongation plastic (%)':'elongation_plastic',
               'REFERENCE: doi':'doi'
               }

In [67]:
extraction_df = pd.concat(extraction_dfs, axis=0).reset_index(drop=True)
# extraction_df.drop(columns=['ultimate tensible strength'], inplace=True)
print(extraction_df.columns)

Index(['formula', 'yield_strength', 'grain_size', 'experimental_density', 'hardness', 'test_temperature', 'ultimate_tensile_strength', 'elongation', 'elongation_plastic', 'evidence', 'doi'], dtype='object')


In [69]:
display(extraction_df.shape)
columns_to_drop = [col for col in extraction_df.columns if col not in colname_map.values()]
extraction_df.drop(columns=columns_to_drop, inplace=True)
display(extraction_df)

(947, 10)

,formula,yield_strength,grain_size,experimental_density,hardness,test_temperature,ultimate_tensile_strength,elongation,elongation_plastic,doi
0,Al 20 Cr 10 Nb 15 Ti 20 V 25 Zr 10,1535.0,0.0,5.55,0.0,22.0,0.0,0.6,0.0,10.1016.j.matlet.2020.127372
1,Al 20 Cr 10 Nb 15 Ti 20 V 25 Zr 10,1000.0,0.0,5.55,0.0,800.0,0.0,0.0,0.0,10.1016.j.matlet.2020.127372
2,Al20Cr10Nb15Ti20V25Zr10,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,10.1016.j.matlet.2020.127372
3,Al 10 Nb 15 Ta 5 Ti 30 Zr 40,1050.0,0.0,0.00,0.0,23.0,0.0,0.0,0.0,10.1016.j.mtla.2020.100627
4,Al 10 Nb 15 Ta 5 Ti 30 Zr 40,852.0,0.0,0.00,0.0,400.0,0.0,0.0,0.0,10.1016.j.mtla.2020.100627
...,...,...,...,...,...,...,...,...,...,...
942,Ti 38 V 15 Nb 23 Hf 24,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,10.1038.s41563-020-0750-4
943,Ti 38 V 15 Nb 23 Hf 24,200.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,10.1038.s41563-020-0750-4
944,MoNbHfZrTi,1719.0,0.0,0.00,0.0,293.0,0.0,0.0,0.0,10.1016.j.matdes.2015.05.019
945,MoNbHfZrTi,1719.0,0.0,0.00,0.0,293.0,0.0,0.0,0.0,10.1016.j.matdes.2015.05.019


In [70]:
import numpy as np
extraction_df.replace(to_replace='No information', value=np.nan, inplace=True)
display(extraction_df.head(3))

,formula,yield_strength,grain_size,experimental_density,hardness,test_temperature,ultimate_tensile_strength,elongation,elongation_plastic,doi
0,Al 20 Cr 10 Nb 15 Ti 20 V 25 Zr 10,1535.0,0.0,5.55,0.0,22.0,0.0,0.6,0.0,10.1016.j.matlet.2020.127372
1,Al 20 Cr 10 Nb 15 Ti 20 V 25 Zr 10,1000.0,0.0,5.55,0.0,800.0,0.0,0.0,0.0,10.1016.j.matlet.2020.127372
2,Al20Cr10Nb15Ti20V25Zr10,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,10.1016.j.matlet.2020.127372


# Aligning extracted and ground-truth papers

In [71]:
paper_df.iloc[0]

IDENTIFIER: Reference ID                                                                  7
FORMULA                                   Al0.025 Cr0.2 Mo0.2 Nb0.2 Ta0.025 Ti0.2 V0.05 ...
PROPERTY: Microstructure                                                          BCC+Laves
PROPERTY: Processing method                                                            CAST
PROPERTY: BCC/FCC/other                                                               other
PROPERTY: grain size ($\mu$m)                                                           NaN
PROPERTY: Exp. Density (g/cm$^3$)                                                      7.44
PROPERTY: Computed Density (g/cm$^3$)                                                   7.5
PROPERTY: HV                                                                            NaN
PROPERTY: Type of test                                                                    C
PROPERTY: Test temperature ($^\circ$C)                                          

In [72]:
extraction_df.iloc[0]

formula                      Al 20 Cr 10 Nb 15 Ti 20 V 25 Zr 10
yield_strength                                           1535.0
grain_size                                                  0.0
experimental_density                                       5.55
hardness                                                    0.0
test_temperature                                           22.0
ultimate_tensile_strength                                   0.0
elongation                                                  0.6
elongation_plastic                                          0.0
doi                                10.1016.j.matlet.2020.127372
Name: 0, dtype: object

## Hardcoded column name mapping

In [73]:
for col in extraction_df.columns:
  assert col in colname_map.values(), print(col)

for col in colname_map.values():
  assert col in extraction_df.columns

for col in colname_map.keys():
  assert col in paper_df.columns, print(col)

inverted_colname_map = {value:key for key,value in colname_map.items()}

In [74]:
extraction_df.columns

Index(['formula', 'yield_strength', 'grain_size', 'experimental_density', 'hardness', 'test_temperature', 'ultimate_tensile_strength', 'elongation', 'elongation_plastic', 'doi'], dtype='object')

## Matching up DOIs

In [75]:
paper_df['REFERENCE: doi']

0      10.1016/j.jallcom.2020.155700
1      10.1016/j.jallcom.2020.155700
2      10.1016/j.jallcom.2020.155700
3      10.1016/j.jallcom.2020.155700
4         10.1016/j.msea.2020.139275
                   ...              
966       10.1016/j.msea.2021.141168
967       10.1016/j.msea.2021.141168
968                10.3390/e21020114
969       10.1016/j.msea.2003.12.011
970       10.1016/j.msea.2003.12.011
Name: REFERENCE: doi, Length: 971, dtype: object

In [76]:
extraction_df['doi']

0      10.1016.j.matlet.2020.127372
1      10.1016.j.matlet.2020.127372
2      10.1016.j.matlet.2020.127372
3        10.1016.j.mtla.2020.100627
4        10.1016.j.mtla.2020.100627
                   ...             
942       10.1038.s41563-020-0750-4
943       10.1038.s41563-020-0750-4
944    10.1016.j.matdes.2015.05.019
945    10.1016.j.matdes.2015.05.019
946    10.1016.j.matdes.2015.05.019
Name: doi, Length: 947, dtype: object

In [77]:
#Should be 20 or so uniques, unless there were papers from which satanu couldn't extract anything
extraction_dois = extraction_df['doi'].unique()
len(extraction_dois)

116

In [78]:
# Two possible ways to find the set of ground-truth records:
# 1: match on DOIs from the extraction dataset
# 2: Grab the top 20 alphabetical paper titles, which is supposedly what Satanu did

# Alphabetical approach not working because the extracted records appear to include the top 21st, 22nd, and 23rd titles,
# but are missing 4 titles that are above them (see below for analysis)

# DOI matching mostly works, but it is unable to find the 20th paper attempted by Satanu, which means that the performance
# will be slightly overestimated, because it will not account for the paper for which, presumably, Satanu's extraction
# wasn't able to find anything

paper_df['processed_doi'] =  paper_df['REFERENCE: doi'].apply(lambda doi:doi.replace('/','.'))

# filtered_paper_df = paper_df[paper_df['processed_doi'].isin(extraction_dois)]

# # So this isn't ideal. Should be 20. Need to get list of 20 dois from Satanu, but will continue onward for now.
# len(filtered_paper_df['REFERENCE: doi'].unique())
first_20_papers = sorted(paper_df['REFERENCE: title'][paper_df['REFERENCE: title'].notnull()].unique())[0:20]
alphabetical_filtered_paper_df = paper_df[paper_df['REFERENCE: title'].isin(first_20_papers)]

doi_match_filtered_paper_df = paper_df[paper_df['processed_doi'].isin(extraction_dois)]

filtered_paper_df =doi_match_filtered_paper_df #Using the DOI matching for now til I resolve things with Satanu

print(filtered_paper_df.shape)
print('Filtered titles:',len(filtered_paper_df['REFERENCE: title'].unique())) # should be 20
print('Filtered DOIs:',len(filtered_paper_df['REFERENCE: doi'].unique())) # should be 20


filtered_paper_dois = filtered_paper_df['REFERENCE: doi'].apply(lambda doi:doi.replace('/','.')).unique()

overlap = set(filtered_paper_dois).intersection(set(extraction_dois))
print('Overlapping DOIs:',len(overlap)) #should be size of smaller set
assert len(overlap) == min(len(filtered_paper_dois), len(extraction_dois))

(818, 24)
Filtered titles: 115
Filtered DOIs: 115
Overlapping DOIs: 115


In [79]:
# Analyze the non-overlap between the alphabetical ground truth papers and extracted papers
processed_filtered_dois = set(alphabetical_filtered_paper_df['processed_doi'].unique())

print('Extracted but not included in the alphabetical filtered paper DF:')
extracted_not_filtered_dois = (set(extraction_dois).difference(processed_filtered_dois))
print(extracted_not_filtered_dois)
display(paper_df[paper_df['processed_doi'].isin(extracted_not_filtered_dois)]['REFERENCE: title'].unique())
# So Satanu's extracted results includes some papers which shouldn't be in the alphabetical top 20


print('\nPresent in alphabetical filtered paper df but not extracted')
filtered_not_extracted = (set(processed_filtered_dois).difference(extraction_dois))
print(filtered_not_extracted)
display(paper_df[paper_df['processed_doi'].isin(filtered_not_extracted)]['REFERENCE: title'].unique())
# And it excludes some papers which should be in the top 20, alphabetically


Extracted but not included in the alphabetical filtered paper DF:
{'10.1016.j.jallcom.2021.159740', '10.1016.j.intermet.2015.09.011', '10.3390.e18050189', '10.1016.j.jallcom.2020.154301', '10.1016.j.scriptamat.2021.114225', '10.1016.j.mtla.2020.100627', '10.1016.j.jallcom.2015.11.091', '10.1016.j.msea.2003.12.011', '10.1016.j.matchemphys.2017.06.054', '10.1016.j.matlet.2016.03.133', '10.1007.s11661-018-4646-8', '10.1016.j.intermet.2015.03.013', '10.1016.j.jallcom.2021.161187', '10.1007.s40195-019-00935-x', '10.1016.j.scriptamat.2021.113919', '10.1016.j.scriptamat.2016.10.028', '10.1016.j.scriptamat.2020.09.027', '10.1016.j.msea.2015.12.017', '10.1016.j.jallcom.2021.159190', '10.1007.s42864-021-00111-8', '10.1016.j.matchar.2021.111287', '10.1016.j.jmst.2020.07.012', '10.1016.j.msea.2021.140798', '10.1016.j.jallcom.2014.11.064', '10.1016.j.msea.2017.10.073', '10.1007.s10853-012-6260-2', '10.1016.j.actamat.2019.06.006', '10.1016.j.ijrmhm.2021.105664', '10.1016.j.ijrmhm.2019.02.009', '10.1

array(['Effects of Al addition on the microstructures and properties of MoNbTaTiV refractory high entropy alloy',
       'Lightweight Zr1.2V0.8NbTi Al  high-entropy alloys with high tensile strength and ductility',
       'The mechanical and oxidation properties of novel B2-ordered Ti2ZrHf0.5VNb0.5Alx refractory high-entropy alloys',
       'Microstructure and Mechanical Properties of VTaTiMoAlsubx/sub Refractory High Entropy Alloys',
       'Microstructures and mechanical properties of Ta‚ÄìNb‚ÄìZr‚ÄìTi‚ÄìAl refractory high entropy alloys with varying Ta/Ti ratios',
       'Effects of Al on Precipitation Behavior of Ti-Nb-Ta-Zr Refractory High Entropy Alloys',
       'Effect of Al addition on mechanical properties and microstructure of refractory AlxHfNbTaTiZr alloys',
       'High temperature strength of refractory complex concentrated alloys',
       'Microstructures and Crackling Noise of AlxNbTiMoV High Entropy Alloys',
       'Microstructure and Compressive Properties of NbTiVTaA


Present in alphabetical filtered paper df but not extracted
{'10.1063.1.4966659', '10.1007.s11661-017-4386-1', '10.1007.s10853-019-03567-9', '10.1016.j.jallcom.2018.11.318'}


array(['Contribution of Lattice Distortion to Solid Solution Strengthening in a Series of Refractory High Entropy Alloys',
       'A fine-grained NbMoTaWVCr refractory high-entropy alloy with ultra-high strength: Microstructural evolution and mechanical properties',
       'Alloy design for intrinsically ductile refractory high-entropy alloys',
       'Correction to: Effect of Si additions on microstructure and mechanical properties of refractory NbTaWMo high-entropy alloys'],
      dtype=object)

In [80]:
first_20_papers

['A Novel Series of Refractory High-Entropy Alloys Ti2ZrHf0.5VNbx with High Specific Yield Strength and Good Ductility',
 'A fine-grained NbMoTaWVCr refractory high-entropy alloy with ultra-high strength: Microstructural evolution and mechanical properties',
 'A new refractory Ti-Nb-Hf-Al high entropy alloy strengthened by orthorhombic phase particles',
 'A novel HfNbTaTiV high-entropy alloy of superior mechanical properties designed on the principle of maximum lattice distortion',
 'A refractory Hf25Nb25Ti25Zr25 high-entropy alloy with excellent structural stability and tensile properties',
 'A simplified model connecting lattice distortion with friction stress of Nb-based equiatomic high-entropy alloys',
 'A single-phase V0.5Nb0.5ZrTi refractory high-entropy alloy with outstanding tensile properties',
 'Alloy Design and Properties Optimization of High-Entropy Alloys',
 'Alloy design for intrinsically ductile refractory high-entropy alloys',
 'An AlNbTiVZr0.5 high-entropy alloy combin

# Aligning extracted and ground-truth rows

In [81]:
# So at this point we have filtered_paper_df, consisting of the ground truth extracted values for the 20-paper sample,
# and extraction_df, consisting of the extracted values. colname_map maps from ground truth col names to extracted col names

In [82]:
target_cols = ['PROPERTY: HV',
               'PROPERTY: YS (MPa)',
               'PROPERTY: UTS (MPa)',
               'PROPERTY: Elongation (%)',
               'PROPERTY: Elongation plastic (%)',
               'PROPERTY: Exp. Young modulus (GPa)']

In [83]:
filtered_paper_df.columns

Index(['IDENTIFIER: Reference ID', 'FORMULA', 'PROPERTY: Microstructure', 'PROPERTY: Processing method', 'PROPERTY: BCC/FCC/other', 'PROPERTY: grain size ($\mu$m)', 'PROPERTY: Exp. Density (g/cm$^3$)', 'PROPERTY: Computed Density (g/cm$^3$)', 'PROPERTY: HV', 'PROPERTY: Type of test',
       'PROPERTY: Test temperature ($^\circ$C)', 'PROPERTY: YS (MPa)', 'PROPERTY: UTS (MPa)', 'PROPERTY: Elongation (%)', 'PROPERTY: Elongation plastic (%)', 'PROPERTY: Exp. Young modulus (GPa)', 'PROPERTY: Computed Young modulus (GPa)', 'PROPERTY: O content (wppm)', 'PROPERTY: N content (wppm)',
       'PROPERTY: C content (wppm)', 'REFERENCE: doi', 'REFERENCE: year', 'REFERENCE: title', 'processed_doi'],
      dtype='object')

In [84]:
filtered_paper_df.iloc[0]

IDENTIFIER: Reference ID                                                                  7
FORMULA                                   Al0.025 Cr0.2 Mo0.2 Nb0.2 Ta0.025 Ti0.2 V0.05 ...
PROPERTY: Microstructure                                                          BCC+Laves
PROPERTY: Processing method                                                            CAST
PROPERTY: BCC/FCC/other                                                               other
PROPERTY: grain size ($\mu$m)                                                           NaN
PROPERTY: Exp. Density (g/cm$^3$)                                                      7.44
PROPERTY: Computed Density (g/cm$^3$)                                                   7.5
PROPERTY: HV                                                                            NaN
PROPERTY: Type of test                                                                    C
PROPERTY: Test temperature ($^\circ$C)                                          

In [85]:
extraction_df.iloc[0]

formula                      Al 20 Cr 10 Nb 15 Ti 20 V 25 Zr 10
yield_strength                                           1535.0
grain_size                                                  0.0
experimental_density                                       5.55
hardness                                                    0.0
test_temperature                                           22.0
ultimate_tensile_strength                                   0.0
elongation                                                  0.6
elongation_plastic                                          0.0
doi                                10.1016.j.matlet.2020.127372
Name: 0, dtype: object

## Unique records

In [86]:
paper_keys = ['FORMULA',
              'PROPERTY: Microstructure',
              'PROPERTY: Processing method',
              'PROPERTY: BCC/FCC/other',
              'PROPERTY: grain size ($\mu$m)',
              'PROPERTY: Type of test',
              'PROPERTY: Test temperature ($^\circ$C)']

extraction_keys = [colname_map[paper_key] for paper_key in paper_keys if paper_key in colname_map]
print(extraction_keys)

['formula', 'grain_size', 'test_temperature']


I think we need to be trying to extract more keys from the data

We'll consider a unique record to be a unique combination of those 7 properties, and we will consider full matches and likely matches.

### normalize_formula() function

In [87]:
import pymatgen.core as mg


In [88]:
f1 = "Hf0.105 Nb0.053 Ti0.421 V0.211 Zr0.211"
f2 = "Ti2ZrHf0.5VNb0.25"


In [89]:
def rounding_fractions(comp_string: str) -> list[str]:
    elements = [f for i, f in enumerate(comp_string.split('$')) if i%2==0]
    fractions = [str(round(float(f.replace("_{", "").replace("}", "")), 3)) for i, f in enumerate(comp_string.split('$')) if
                 i % 2 != 0]
    final_results = [f"{i}{j}" for i, j in zip(elements, fractions)]
    return final_results

def normalize_comp_name(compName: str) -> str:
  try:
    comp = mg.Composition(compName)
    comp = mg.Composition(comp.alphabetical_formula)
    fractional_comp = comp.fractional_composition
    fract_comp_ustring = fractional_comp.to_unicode_string()
    normalized_frac_comp = rounding_fractions(fract_comp_ustring)
    normalized_comp_name = ''.join(f for f in normalized_frac_comp)

    return normalized_comp_name
  except:
    print(compName)
    return None


display(normalize_comp_name(f1))
display(normalize_comp_name(f2))

'Hf0.105Nb0.053Ti0.421V0.211Zr0.211'

'Hf0.105Nb0.053Ti0.421V0.211Zr0.211'

In [90]:
from traceback import print_exc
import re

float_pattern = '[0-9]*\.?[0-9]+'
def round_string_floats(s:str, precision:int=2):
  floats = re.search(float_pattern, s)


def normalize_formula(formula):
  try:
    return mg.Composition(formula).get_integer_formula_and_factor()[0]
  except Exception as ex:
    # print_exc()
    return None

f1 = "Hf0.105 Nb0.053 Ti0.421 V0.211 Zr0.211"
f2 = "Ti2ZrHf0.5VNb0.25"

display(normalize_formula(f1))
display(normalize_formula(f2))

round_string_floats(f1)

'Hf105Zr211Ti421Nb53V211'

'Hf2Zr4Ti8NbV4'

In [91]:
c1 = mg.Composition(f1)
display(c1)
c2 = mg.Composition(f2)
display(c2.fractional_composition)

display(c1.almost_equals(c2,rtol=0.1, atol=0.1))

Comp: Hf0.105 Zr0.211 Ti0.421 Nb0.053 V0.211

Comp: Hf0.10526316 Zr0.21052632 Ti0.42105263 Nb0.05263158 V0.21052632

False

In [92]:
c2 = mg.Composition(f2)
c2.fractional_composition

Comp: Hf0.10526316 Zr0.21052632 Ti0.42105263 Nb0.05263158 V0.21052632

In [93]:
# filtered_paper_df['normalized_formula'] = filtered_paper_df['FORMULA'].apply(normalize_formula)
filtered_paper_df['normalized_formula'] = filtered_paper_df['FORMULA'].apply(normalize_comp_name)

filtered_paper_df.sort_values('processed_doi')[['FORMULA','normalized_formula','processed_doi']]

/tmp/ipykernel_1793644/2496855811.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_paper_df['normalized_formula'] = filtered_paper_df['FORMULA'].apply(normalize_comp_name)


,FORMULA,normalized_formula,processed_doi
355,Fe0.167 Mo0.167 Ni0.167 Ti0.167 V0.167 Zr0.167,Fe0.167Mo0.167Ni0.167Ti0.167V0.167Zr0.167,10.1002.adem.200300567
823,Mo0.25 Re0.25 Ta0.25 W0.25,Mo0.25Re0.25Ta0.25W0.25,10.1002.adem.202100765
822,Mo0.25 Re0.25 Ta0.25 W0.25,Mo0.25Re0.25Ta0.25W0.25,10.1002.adem.202100765
622,Hf0.25 Ta0.25 Ti0.25 Zr0.25,Hf0.25Ta0.25Ti0.25Zr0.25,10.1002.adma.201701678
627,Hf0.278 Ta0.167 Ti0.278 Zr0.278,Hf0.278Ta0.167Ti0.278Zr0.278,10.1002.adma.201701678
...,...,...,...
424,Hf0.182 Nb0.182 Si0.091 Ti0.182 V0.182 Zr0.182,Hf0.182Nb0.182Si0.091Ti0.182V0.182Zr0.182,10.4028.www.scientific.net.MSF.849.76
11,Al0.048 Mo0.238 Ta0.238 Ti0.238 V0.238,Al0.048Mo0.238Ta0.238Ti0.238V0.238,10.4028.www.scientific.net.MSF.898.638
120,Al0.13 Mo0.217 Ta0.217 Ti0.217 V0.217,Al0.13Mo0.217Ta0.217Ti0.217V0.217,10.4028.www.scientific.net.MSF.898.638
824,Mo0.25 Ta0.25 Ti0.25 V0.25,Mo0.25Ta0.25Ti0.25V0.25,10.4028.www.scientific.net.MSF.898.638


In [96]:
# extraction_df['normalized_formula'] = extraction_df['high_entropy_alloy_formula'].apply(normalize_formula)
extraction_df['normalized_formula'] = extraction_df['formula'].apply(normalize_comp_name)

extraction_df.sort_values('doi')[['formula','normalized_formula','doi']]

Al-containing refractory HEA
AlNb-TaTi
RHEAs
AlxNbTiMoV
Al x NbTiMoV
(Ta 25-x Nb 25 Zr 25 Ti 25+x ) 95 Al 5
Ta 25-x Nb 25 Zr 25 Ti 25+x 95 Al 5
AlxHfNbTaTiZr
AlxHfNbTaTiZr
CrNbTIZr
TiZrNbMoVx
CoCrFeNiAlNb x
S < -θ -τ ext ˙ γ t DSA
S < -θ -τ ext ˙ γ t DSA
S < -θ -τ ext ˙ γ t DSA
CuCoNiCrAl x Fe
Hf-27Ta
Hf-21Nb-21Ta
Hf-21Mo-21Ta
Hf-21Ta-21W
Hf-21Ta-21W
Hf-27Ta
Hf-21Mo-21Ta
Hf-21Nb-21Ta
Hf-21Ta-21W
HfMo x NbTaTiZr
HfMo x NbTaTiZr
A l 0.5 Mo 0.5 NbTa 0.5 TiZr
A l 0.25 NbTaTiZr
AlxNbTaTiV
HfMo 0.5 NbTiV 0.5 Si x
MoNbRe 0.5 W(TaC) x
MoNbRe0.5W(TaC)x
MoNbRe0.5W(TaC)x
MoNbRe0.5W(TaC)x
MoNbRe0.5W(TaC)x
MoNbRe0.5W(TaC)x
MoNbRe0.5W(TaC)x
HEA1200
HEA800
Ti2ZrHf0.5VNb0.5Alx
Ti2ZrHf0.5VNb0.5Alx
Ti2ZrHf0.5VNb0.5Alx
Ti2ZrHf0.5VNb0.5Alx
Ti2ZrHf0.5VNb0.5Alx
Ti2ZrHf0.5VNb0.5Alx
Ti2ZrHf0.5VNb0.5Alx
Ti2ZrHf0.5V0.5NbAlx
Ti2ZrHf0.5VNb0.5Alx
Ti2ZrHf0.5V0.5NbAlx
TiNbTa0.5ZrAlx
RHEA-Al0
RHEA-Al0.2
RHEA-Al0.5
RHEA-Al1.0
TiNbTa0.5ZrAlx
TiNbTa0.5ZrAlx
Ti-Nb-Ta-Zr-Al
Ti-6Cr-5Mo-5V-4Al
HEA
(NbTaW)1-xMox
(NbTaW)1-xMo

,formula,normalized_formula,doi
201,CuCoNiCrAlFeMoTiVZr,Al0.1Co0.1Cr0.1Cu0.1Fe0.1Mo0.1Ni0.1Ti0.1V0.1Zr0.1,10.1002.adem.200300567
202,CuCoNiCrAl x Fe,None,10.1002.adem.200300567
203,CuCoNiCrAl 0.5 Fe,Al0.091Co0.182Cr0.182Cu0.182Fe0.182Ni0.182,10.1002.adem.200300567
204,CuCoNiCrAlFeTiV,Al0.125Co0.125Cr0.125Cu0.125Fe0.125Ni0.125Ti0....,10.1002.adem.200300567
205,Cu 0.5 CoNiCrAl,Al0.222Co0.222Cr0.222Cu0.111Ni0.222,10.1002.adem.200300567
...,...,...,...
484,VTaTiMoAl0,Mo0.25Ta0.25Ti0.25V0.25,10.4028.www.scientific.net.MSF.898.638
485,VTaTiMoAl0.2,Al0.048Mo0.238Ta0.238Ti0.238V0.238,10.4028.www.scientific.net.MSF.898.638
486,VTaTiMoAl0.6,Al0.13Mo0.217Ta0.217Ti0.217V0.217,10.4028.www.scientific.net.MSF.898.638
487,VTaTiMoAl1,Al0.2Mo0.2Ta0.2Ti0.2V0.2,10.4028.www.scientific.net.MSF.898.638


Basic logic: for each unique record in ground truth data, try to locate that record in the extracted data, as either:

Exact match between unique keys

Partial match between unique keys, with no other possible candidates

In [97]:
for target_col in target_cols:
  print('Target:',target_col)

Target: PROPERTY: HV
Target: PROPERTY: YS (MPa)
Target: PROPERTY: UTS (MPa)
Target: PROPERTY: Elongation (%)
Target: PROPERTY: Elongation plastic (%)
Target: PROPERTY: Exp. Young modulus (GPa)


## Row match functions

### assess_row_row_match()

In [98]:
import pandas as pd


from typing import Sequence, Dict, List


def assess_row_row_match(data_row: pd.Series,
                         extracted_row: pd.Series,
                         required_keys: Sequence[str],
                         other_keys: Sequence[str],
                         ):
    '''
    Assess the extent to which an extracted row matches a dataset row.
    Full if it matches all required and other keys
    Partial if it matches required keys and at least doesn't violate other keys (i.e. it has nulls where they have values)
    Nonmatch if it doesn't match required keys, or if violates any other keys (conflicting values)

    If we want to add flexible matching or unit conversion or something,
    (e.g. making '100mL' match '0.1L'), this is where to put it

    But nothing like this is currently implemented
    '''
    all_keys = required_keys + other_keys
    match_strength = 0
    for key in all_keys:
        if type(data_row[key]) == float and type(extracted_row[key]) == str:
          try:
            extracted_row[key] = float(extracted_row[key])
          except:
            pass
        if key in required_keys and extracted_row[key] != data_row[key]:
            return 0.0
        elif extracted_row[key] == data_row[key] or pd.isnull(extracted_row[key]) and pd.isnull(data_row[key]):
            match_strength +=1

    match_strength /= len(all_keys)
    return match_strength
    return result

In [99]:
r1 = filtered_paper_df.loc[375:375]
display(r1)

,IDENTIFIER: Reference ID,FORMULA,PROPERTY: Microstructure,PROPERTY: Processing method,PROPERTY: BCC/FCC/other,PROPERTY: grain size ($\mu$m),PROPERTY: Exp. Density (g/cm$^3$),PROPERTY: Computed Density (g/cm$^3$),PROPERTY: HV,PROPERTY: Type of test,PROPERTY: Test temperature ($^\circ$C),PROPERTY: YS (MPa),PROPERTY: UTS (MPa),PROPERTY: Elongation (%),PROPERTY: Elongation plastic (%),PROPERTY: Exp. Young modulus (GPa),PROPERTY: Computed Young modulus (GPa),PROPERTY: O content (wppm),PROPERTY: N content (wppm),PROPERTY: C content (wppm),REFERENCE: doi,REFERENCE: year,REFERENCE: title,processed_doi,normalized_formula
375,142,Hf0.105 Nb0.053 Ti0.421 V0.211 Zr0.211,BCC,CAST,BCC,NaN,NaN,6.4,332.4,C,25.0,1115.0,NaN,50.0,NaN,NaN,104.0,NaN,NaN,NaN,10.1007/s40195-019-00921-3,2019.0,A Novel Series of Refractory High-Entropy Allo...,10.1007.s40195-019-00921-3,Hf0.105Nb0.053Ti0.421V0.211Zr0.211


In [100]:
r2 = extraction_df.loc[1:1].rename(columns=inverted_colname_map)
display(r2)

,FORMULA,PROPERTY: YS (MPa),PROPERTY: grain size ($\mu$m),PROPERTY: Exp. Density (g/cm$^3$),PROPERTY: HV,PROPERTY: Test temperature ($^\circ$C),PROPERTY: UTS (MPa),PROPERTY: Elongation (%),PROPERTY: Elongation plastic (%),REFERENCE: doi,normalized_formula
1,Al 20 Cr 10 Nb 15 Ti 20 V 25 Zr 10,1000.0,0.0,5.55,0.0,800.0,0.0,0.0,0.0,10.1016.j.matlet.2020.127372,Al0.2Cr0.1Nb0.15Ti0.2V0.25Zr0.1


In [101]:
assess_row_row_match(data_row=r1.iloc[0],
                         extracted_row=r2.iloc[0],
                         required_keys=['normalized_formula', 'PROPERTY: YS (MPa)'],
                         other_keys= [
                            #  'PROPERTY: Microstructure',
              # 'PROPERTY: Processing method',
              # 'PROPERTY: BCC/FCC/other',
              'PROPERTY: grain size ($\mu$m)',
              # 'PROPERTY: Type of test',
              'PROPERTY: Test temperature ($^\circ$C)'])

0.0

### assess_df_df_match()

In [102]:
from typing import List, Tuple

def assess_df_df_matches(
        data_df: pd.DataFrame,  # test df
        extracted_df: pd.DataFrame,  # extracted df
        required_keys: List[str],
        other_keys: List[str]
):
    '''
    Assess pairwise row matches between two dfs. Assume columns match.
    '''
    extracted_data_matches = {}

    for extracted_idx, extracted_row in extracted_df.iterrows():
        extracted_data_matches[extracted_idx] = {}
        for data_idx, data_row in data_df.iterrows():

            row_match = assess_row_row_match(data_row=data_row,
                                             extracted_row=extracted_row,
                                             required_keys=required_keys, other_keys=other_keys)
            extracted_data_matches[extracted_idx][data_idx] = row_match

    return extracted_data_matches


### perform_matching()

In [103]:
def perform_matching(extracted_data_matches:Dict[int, Dict[int,float]], exclusive_threshold=0.2, verbose:bool=False):
    '''
    Given a dictionary of row-to-row match strength values, determine a likely one-to-one pairing of rows
    by greedily picking the strongest match until there are no more unmatched rows or until no matches are stronger
    than the threshold value

    :param extracted_data_matches:
    :param data_extracted_matches:
    :return:
    '''
    unmatched_extracted_idxs = set()
    unmatched_data_idxs = set()
    match_strengths = []

    for extracted_idx, match_dict in extracted_data_matches.items():
        for data_idx, match_strength in match_dict.items():
            match_strengths.append((extracted_idx, data_idx, match_strength))
            unmatched_extracted_idxs.add(extracted_idx)
            unmatched_data_idxs.add(data_idx)

    match_strengths = sorted(match_strengths, key=lambda m:m[2], reverse=True)
    if verbose:
      print(match_strengths)

    matches= []

    for extracted_idx, data_idx, match_strength in match_strengths:
        if match_strength > exclusive_threshold and extracted_idx in unmatched_extracted_idxs and data_idx in unmatched_data_idxs:
            matches.append((extracted_idx, data_idx, match_strength))
            unmatched_extracted_idxs.remove(extracted_idx)
            unmatched_data_idxs.remove(data_idx)

    return matches, list(unmatched_extracted_idxs), list(unmatched_data_idxs)


### align_dfs()

In [104]:
from typing import Union, Callable
def align_dfs(data_df: pd.DataFrame,  # test df
                      extracted_df: pd.DataFrame,  # extracted df
                      required_keys: List[str],  # keys rows absolutely must match on to be a potential match (e.g. formula, target value (yield strength, etc))
                      other_keys: List[str],  # keys rows could/should match on if possible, but a non-match isn't disqualifying (e.g. grain size, test temp)
                      extracted_to_data_colmap: Dict[str, str],  # map between data and extracted df columns
                      group_by: Union[str, Tuple[str,str]],  # group by this before doing the matching. Should be paper title or DOI. If tuple, first is for data_df, second for extracted_df
                      discard_nonextracted_keys: bool = False,  # If true, ignore other_keys that don't exist in the extracted_df
                      verbose: bool = False,
                      debug_group_count: int=None,
                      debug_group_skip:int=None,
                      debug_group_filter:Callable=None
                      ):
    '''
    :param data_df: human-annotated df
   :param extracted_df: model-extracted df. Should have same structure as human-annotated one, but columns don't have to match
   :param required_keys: keys that absolutely have to match for a pair of records to be considered a potential match (e.g. formula and YS, if YS is the target value)
   :param other_keys: keys that should match, but don't absolutely have to (e.g. grain size, test temp)
   :param extracted_to_data_colmap: dictionary mapping column names from extracted DF to original DF
   :param group_by: column to group by before doing the matching. Should be title or DOI
   :param discard_nonextracted_keys: whether to ignore any keys that weren't extracted, as these could never be a match (e.g. test type if we didn't try to extract it)
   :param verbose: whether to print additional output
   :return:

    Take a dataframe of human-extracted values (e.g. Carolina's dataset), and align it with
    a dataframe of model-extracted values.

    Goal here is alignment--figure out which extracted records match with which original records,
    and which original and extracted records are unmatched.

    Do this with a greedy matching algorithm.

    Return a combined dataframe where matched records are merged side by side with their columns
    suffixed with '_original' and '_extracted' respectively. Unmatched records are included too, with
    NaN values for the columns that couldn't be matched.

   index  extracted_idx  original_idx  match_strength match_group formula_original  test_temp_original  grain_size_original test_type_original  ys_original title_original formula_extracted  test_temp_extracted grain_size_extracted  ys_extracted title_extracted
0      0            0.0           0.0            0.75     paper_1              NbV               100.0                 25.5             Normal         14.4        paper_1               NbV                100.0                 None          14.4         paper_1
1      1            NaN           1.0             NaN     paper_1              NbV               200.0                 34.0             Normal         24.8        paper_1               NaN                  NaN                  NaN           NaN             NaN
2      1            1.0           NaN             NaN     paper_1              NaN                 NaN                  NaN                NaN          NaN            NaN               NbV                300.0                 None          24.5         paper_1

    So when you want to isolate rows that were successfully matched, just look for pd.notnull(original_idx) and pd.notnull(extracted_idx), and
    vice versa for finding original or extracted rows that couldn't be matched.

    '''
    all_keys = required_keys + other_keys
    data_keys = [col for col in all_keys if col in data_df]

    if verbose:
        print('True data DF:')
        display(data_df[data_keys])

    extracted_df = extracted_df.rename(columns=extracted_to_data_colmap)

    if discard_nonextracted_keys:
        other_keys = [key for key in other_keys if key in extracted_df]

    extracted_keys  = [key for key in all_keys if key in extracted_df]

    if verbose:
        print('Extracted data DF (with renamed columns):')
        display(extracted_df[extracted_keys])
        print('Required keys:', required_keys)
        print('Other keys to match on:', other_keys)
        print('\nMatching rows:')

    if type(group_by) == str:
      data_groups = data_df.groupby(group_by)
      extracted_groups = extracted_df.groupby(group_by)
    elif type(group_by) == tuple:
      data_groups = data_df.groupby(group_by[0])
      extracted_groups = extracted_df.groupby(group_by[1])
      group_by=group_by[0]

    # By definition we can assume that data_df has a nonempty group for every paper
    # The same is not necessarily true for extracted_df, if no values were found for a paper

    match_dfs = []
    group_count = 0
    group_skip = 0 if debug_group_skip is None else debug_group_skip

    for group, data_indices in data_groups.indices.items():
        extracted_indices = extracted_groups.indices[group] if group in extracted_groups.indices else []

        if verbose:
          print('---')
          print(group_by, group)
          print('original indices',data_indices)
          print('extracted indices', extracted_indices)

        group_data_df = data_df.iloc[data_indices]
        group_extracted_df = extracted_df.iloc[extracted_indices]

        if debug_group_filter and ( not debug_group_filter(group_data_df) or not debug_group_filter(group_extracted_df)):
          if verbose: print('Group debug filter fail. Skipping.')
          continue

        group_count +=1
        if group_count <= group_skip:
          if verbose:print('Skipping for debug')
          continue

        if verbose:
          print('Group data df:')
          display(group_data_df[data_keys])
          print('Group extracted df:')
          display(group_extracted_df[extracted_keys])

        e_d_matches = assess_df_df_matches(data_df=group_data_df,
                                                        extracted_df=group_extracted_df,
                                                        required_keys=required_keys,
                                                        other_keys=other_keys)

        # print('extracted matches')
        # print(e_d_matches)

        matches, unmatched_extracted_idxs, unmatched_data_idxs = perform_matching(extracted_data_matches=e_d_matches, verbose=verbose)


        match_df = pd.DataFrame(matches, columns=['extracted_idx','original_idx', 'match_strength'])
        match_df['match_group'] = group

        if verbose:
          print('matches:')
          print(matches)
          print('match df:')
          display(match_df)


        match_df = match_df.merge(group_data_df.rename(columns=lambda c:c+'_original'), left_on='original_idx', right_index=True)

        if verbose:
          print('merged match df:')
          display(match_df)

        match_df = match_df.merge(group_extracted_df.rename(columns=lambda c: c + '_extracted'), left_on='extracted_idx', right_index=True)

        if verbose:
          print('second merged match df:')
          display(match_df)

        unmatched_extracted_df = group_extracted_df.loc[unmatched_extracted_idxs].rename(columns=lambda c:c+'_extracted')
        unmatched_extracted_df['extracted_idx'] = unmatched_extracted_idxs
        unmatched_extracted_df['match_group'] = group

        unmatched_data_df = group_data_df.loc[unmatched_data_idxs].rename(columns=lambda c:c+'_original')
        unmatched_data_df['original_idx'] = unmatched_data_idxs
        unmatched_data_df['match_group'] = group

        match_df = pd.concat([match_df, unmatched_data_df, unmatched_extracted_df], axis=0)
        match_dfs.append(match_df)
        pass

        if verbose:
          print('===')

        if debug_group_count and group_count >= debug_group_count+group_skip:
          break

    all_match_df = pd.concat(match_dfs, axis=0).reset_index()
    return all_match_df


### Testing

#### Toy dataset

In [105]:
required_keys = ['formula', 'ys']
other_keys = [ 'test_temp', 'grain_size', 'test_type']

t_tdf = pd.DataFrame([{'formula':'NbV', 'test_temp':100, 'grain_size':25.5, 'test_type':'Normal', 'ys':14.4, 'title':'paper_1'},
                     {'formula':'NbV', 'test_temp':200, 'grain_size':34, 'test_type':'Normal', 'ys':24.8, 'title':'paper_1'},
                     ])


tkey_mapping = {'test_temp':'test temperature', 'grain_size':'grain size', 'ys':'yield strength'}
inverted_tkey_mapping = {value:key for key, value in tkey_mapping.items()}

t_edf1 = pd.DataFrame([{'formula':'NbV', 'test temperature':100, 'grain size':None, 'yield strength':14.4, 'title':'paper_1'},
                       {'formula':'NbV', 'test temperature':300, 'grain size':None, 'yield strength':24.5, 'title':'paper_1'},
                       ])




df_result = align_dfs(data_df=t_tdf,
                  extracted_df=t_edf1,
                  required_keys=required_keys,
                  other_keys=other_keys,
                  extracted_to_data_colmap=inverted_tkey_mapping,
                  group_by='title',
                  discard_nonextracted_keys=True,
                  verbose=True
                  )

pd.options.display.width=300
print(df_result)

True data DF:


,formula,ys,test_temp,grain_size,test_type
0,NbV,14.4,100,25.5,Normal
1,NbV,24.8,200,34.0,Normal


Extracted data DF (with renamed columns):


,formula,ys,test_temp,grain_size
0,NbV,14.4,100,None
1,NbV,24.5,300,None


Required keys: ['formula', 'ys']
Other keys to match on: ['test_temp', 'grain_size']

Matching rows:
---
title paper_1
original indices [0 1]
extracted indices [0 1]
Group data df:


...

merged match df:


,extracted_idx,original_idx,match_strength,match_group,formula_original,test_temp_original,grain_size_original,test_type_original,ys_original,title_original
0,0,0,0.75,paper_1,NbV,100,25.5,Normal,14.4,paper_1


second merged match df:


,extracted_idx,original_idx,match_strength,match_group,formula_original,test_temp_original,grain_size_original,test_type_original,ys_original,title_original,formula_extracted,test_temp_extracted,grain_size_extracted,ys_extracted,title_extracted
0,0,0,0.75,paper_1,NbV,100,25.5,Normal,14.4,paper_1,NbV,100,None,14.4,paper_1


===
   index  extracted_idx  original_idx  match_strength match_group formula_original  test_temp_original  grain_size_original test_type_original  ys_original title_original formula_extracted  test_temp_extracted grain_size_extracted  ys_extracted title_extracted
0      0            0.0           0.0            0.75     paper_1              NbV               100.0                 25.5             Normal         14.4        paper_1               NbV                100.0                 None          14.4         paper_1
1      1            NaN           1.0             NaN     paper_1              NbV               200.0                 34.0             Normal         24.8        paper_1               NaN                  NaN                  NaN           NaN             NaN
2      1            1.0           NaN             NaN     paper_1              NaN                 NaN                  NaN                NaN          NaN            NaN               NbV                300.0    

#### Real data

In [106]:
# These have to be a match
required_keys = ['normalized_formula', # I added this to both dfs up above in the unique records section
                 'PROPERTY: YS (MPa)']

# These should be a match but it isn't the end of the world if they aren't
other_keys =[ 'PROPERTY: Microstructure',
              'PROPERTY: Processing method',
              'PROPERTY: BCC/FCC/other',
              'PROPERTY: grain size ($\mu$m)',
              'PROPERTY: Type of test',
              'PROPERTY: Test temperature ($^\circ$C)']


df_result = align_dfs(data_df=filtered_paper_df,
                  extracted_df=extraction_df,
                  required_keys=required_keys,
                  other_keys=other_keys,
                  extracted_to_data_colmap=inverted_colname_map,
                  group_by=('processed_doi', 'REFERENCE: doi'),
                  discard_nonextracted_keys=True,
                  # verbose=True,
                  # debug_group_count=1,
                  # debug_group_skip=1,
                  # debug_group_filter = lambda df: np.any(df['PROPERTY: YS (MPa)'].notnull())
                  )



display(df_result)

,index,extracted_idx,original_idx,match_strength,match_group,IDENTIFIER: Reference ID_original,FORMULA_original,PROPERTY: Microstructure_original,PROPERTY: Processing method_original,PROPERTY: BCC/FCC/other_original,PROPERTY: grain size ($\mu$m)_original,PROPERTY: Exp. Density (g/cm$^3$)_original,PROPERTY: Computed Density (g/cm$^3$)_original,PROPERTY: HV_original,PROPERTY: Type of test_original,PROPERTY: Test temperature ($^\circ$C)_original,PROPERTY: YS (MPa)_original,PROPERTY: UTS (MPa)_original,PROPERTY: Elongation (%)_original,PROPERTY: Elongation plastic (%)_original,PROPERTY: Exp. Young modulus (GPa)_original,PROPERTY: Computed Young modulus (GPa)_original,PROPERTY: O content (wppm)_original,PROPERTY: N content (wppm)_original,PROPERTY: C content (wppm)_original,REFERENCE: doi_original,REFERENCE: year_original,REFERENCE: title_original,processed_doi_original,normalized_formula_original,FORMULA_extracted,PROPERTY: YS (MPa)_extracted,PROPERTY: grain size ($\mu$m)_extracted,PROPERTY: Exp. Density (g/cm$^3$)_extracted,PROPERTY: HV_extracted,PROPERTY: Test temperature ($^\circ$C)_extracted,PROPERTY: UTS (MPa)_extracted,PROPERTY: Elongation (%)_extracted,PROPERTY: Elongation plastic (%)_extracted,REFERENCE: doi_extracted,normalized_formula_extracted
0,355,NaN,355.0,NaN,10.1002.adem.200300567,15.0,Fe0.167 Mo0.167 Ni0.167 Ti0.167 V0.167 Zr0.167,BCC+Sec.,CAST,other,NaN,NaN,7.4,740.0,NaN,25.0,NaN,NaN,NaN,NaN,NaN,175.0,NaN,NaN,NaN,10.1002/adem.200300567,2004.0,Nanostructured High-Entropy Alloys with Multip...,10.1002.adem.200300567,Fe0.167Mo0.167Ni0.167Ti0.167V0.167Zr0.167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201,201.0,NaN,NaN,10.1002.adem.200300567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CuCoNiCrAlFeMoTiVZr,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,10.1002.adem.200300567,Al0.1Co0.1Cr0.1Cu0.1Fe0.1Mo0.1Ni0.1Ti0.1V0.1Zr0.1
2,202,202.0,NaN,NaN,10.1002.adem.200300567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CuCoNiCrAl x Fe,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,10.1002.adem.200300567,None
3,203,203.0,NaN,NaN,10.1002.adem.200300567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CuCoNiCrAl 0.5 Fe,0.0,0.0,0.0,0.0,800.0,0.0,0.00,0.0,10.1002.adem.200300567,Al0.091Co0.182Cr0.182Cu0.182Fe0.182Ni0.182
4,204,204.0,NaN,NaN,10.1002.adem.200300567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CuCoNiCrAlFeTiV,0.0,0.8,0.0,0.0,0.0,0.0,0.00,0.0,10.1002.adem.200300567,Al0.125Co0.125Cr0.125Cu0.125Fe0.125Ni0.125Ti0....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1550,1,485.0,11.0,0.5,10.4028.www.scientific.net.MSF.898.638,86.0,Al0.048 Mo0.238 Ta0.238 Ti0.238 V0.238,BCC,CAST,BCC,NaN,NaN,9.1,NaN,C,25.0,1021.0,1249.0,NaN,NaN,NaN,184.0,NaN,NaN,NaN,10.4028/www.scientific.net/MSF.898.638,2017.0,Microstructure and Mechanical Properties of VT...,10.4028.www.scientific.net.MSF.898.638,Al0.048Mo0.238Ta0.238Ti0.238V0.238,VTaTiMoAl0.2,1021.0,0.0,0.0,450.0,0.0,1249.0,6.81,0.0,10.4028.www.scientific.net.MSF.898.638,Al0.048Mo0.238Ta0.238Ti0.238V0.238
1551,2,486.0,120.0,0.5,10.4028.www.scientific.net.MSF.898.638,86.0,Al0.13 Mo0.217 Ta0.217 Ti0.217 V0.217,BCC,CAST,BCC,NaN,NaN,8.5,NaN,C,25.0,962.0,1033.0,NaN,NaN,NaN,174.0,NaN,NaN,NaN,10.4028/www.scientific.net/MSF.898.638,2017.0,Microstructure and Mechanical Properties of VT...,10.4028.www.scientific.net.MSF.898.638,Al0.13Mo0.217Ta0.217Ti0.217V0.217,VTaTiMoAl0.6,962.0,0.0,0.0,483.3,0.0,1033.0,4.20,0.0,10.4028.www.scientific.net.MSF.898.638,Al0.13Mo0.217Ta0.217Ti0.217V0.217
1552,196,NaN,196.0,NaN,10.4028.www.scientific.net.MSF.898.638,86.0,Al0.2 Mo0.2 Ta0.2 Ti0.2 V0.2,BCC,CAST,BCC,NaN,NaN,8.0,NaN,C,25.0,NaN,735.0,NaN,NaN,NaN,166.0,NaN,NaN,NaN,10.4028/www.scientific.net/MSF.898.638,201

##### Looking at matched rows

In [107]:
match_df = df_result[df_result['original_idx'].notnull() & df_result['extracted_idx'].notnull()]
print(match_df.shape)
display(match_df)

(192, 41)


,index,extracted_idx,original_idx,match_strength,match_group,IDENTIFIER: Reference ID_original,FORMULA_original,PROPERTY: Microstructure_original,PROPERTY: Processing method_original,PROPERTY: BCC/FCC/other_original,PROPERTY: grain size ($\mu$m)_original,PROPERTY: Exp. Density (g/cm$^3$)_original,PROPERTY: Computed Density (g/cm$^3$)_original,PROPERTY: HV_original,PROPERTY: Type of test_original,PROPERTY: Test temperature ($^\circ$C)_original,PROPERTY: YS (MPa)_original,PROPERTY: UTS (MPa)_original,PROPERTY: Elongation (%)_original,PROPERTY: Elongation plastic (%)_original,PROPERTY: Exp. Young modulus (GPa)_original,PROPERTY: Computed Young modulus (GPa)_original,PROPERTY: O content (wppm)_original,PROPERTY: N content (wppm)_original,PROPERTY: C content (wppm)_original,REFERENCE: doi_original,REFERENCE: year_original,REFERENCE: title_original,processed_doi_original,normalized_formula_original,FORMULA_extracted,PROPERTY: YS (MPa)_extracted,PROPERTY: grain size ($\mu$m)_extracted,PROPERTY: Exp. Density (g/cm$^3$)_extracted,PROPERTY: HV_extracted,PROPERTY: Test temperature ($^\circ$C)_extracted,PROPERTY: UTS (MPa)_extracted,PROPERTY: Elongation (%)_extracted,PROPERTY: Elongation plastic (%)_extracted,REFERENCE: doi_extracted,normalized_formula_extracted
8,0,488.0,823.0,0.75,10.1002.adem.202100765,5.0,Mo0.25 Re0.25 Ta0.25 W0.25,BCC,CAST,BCC,NaN,NaN,16.8,NaN,C,1600.0,172.0,NaN,NaN,NaN,NaN,347.0,NaN,NaN,NaN,10.1002/adem.202100765,2021.0,WReTaMo Refractory Highâ€Entropy Alloy with H...,10.1002.adem.202100765,Mo0.25Re0.25Ta0.25W0.25,WReTaMo,172.0,0.0,0.00,654.0,1600.0,244.0,25.00,0.0,10.1002.adem.202100765,Mo0.25Re0.25Ta0.25W0.25
23,0,47.0,639.0,0.50,10.1007.s00339-019-2484-1,15.0,Hf0.333 Ti0.333 Zr0.333,Other,CAST,other,NaN,NaN,8.1,211.0,C,25.0,773.0,NaN,17.0,NaN,NaN,87.0,NaN,NaN,NaN,10.1007/s00339-019-2484-1,2019.0,Effect of Sc and Y addition on the microstruct...,10.1007.s00339-019-2484-1,Hf0.333Ti0.333Zr0.333,TiZrHf,773.0,0.0,0.00,211.0,0.0,1184.0,17.00,0.0,10.1007.s00339-019-2484-1,Hf0.333Ti0.333Zr0.333
24,1,48.0,621.0,0.50,10.1007.s00339-019-2484-1,15.0,Hf0.25 Sc0.25 Ti0.25 Zr0.25,Other,CAST,other,NaN,NaN,6.8,233.0,C,25.0,1001.0,NaN,21.9,NaN,NaN,84.0,NaN,NaN,NaN,10.1007/s00339-019-2484-1,2019.0,Effect of Sc and Y addition on the microstruct...,10.1007.s00339-019-2484-1,Hf0.25Sc0.25Ti0.25Zr0.25,TiZrHfSc,1001.0,0.0,0.00,233.0,0.0,1800.0,21.90,0.0,10.1007.s00339-019-2484-1,Hf0.25Sc0.25Ti0.25Zr0.25
25,2,49.0,625.0,0.50,10.1007.s00339-019-2484-1,15.0,Hf0.25 Ti0.25 Y0.25 Zr0.25,Other,CAST,other,NaN,NaN,7.2,241.0,C,25.0,554.0,NaN,17.7,NaN,NaN,82.0,NaN,NaN,NaN,10.1007/s00339-019-2484-1,2019.0,Effect of Sc and Y addition on the microstruct...,10.1007.s00339-019-2484-1,Hf0.25Ti0.25Y0.25Zr0.25,TiZrHfY,554.0,0.0,0.00,241.0,0.0,1071.0,17.70,0.0,10.1007.s00339-019-2484-1,Hf0.25Ti0.25Y0.25Zr0.25
26,3,50.0,569.0,0.50,10.1007.s00339-019-2484-1,15.0,Hf0.2 Sc0.2 Ti0.2 Y0.2 Zr0.2,Other,CAST,other,NaN,NaN,6.4,256.0,C,25.0,793.0,NaN,15.7,NaN,NaN,80.0,NaN,NaN,NaN,10.1007/s00339-019-2484-1,2019.0,Effect of Sc and Y addition on the microstruct...,10.1007.s00339-019-2484-1,Hf0.2Sc0.2Ti0.2Y0.2Zr0.2,TiZrHfScY,793.0,0.0,0.00,256.0,0.0,1365.0,15.70,0.0,10.1007.s00339-019-2484-1,Hf0.2Sc0.2Ti0.2Y0.2Zr0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1541,0,769.0,422.0,0.50,10.4028.www.scientific.net.MSF.849.76,78.0,Hf0.182 Nb0.182 Si0.091 Ti0.182 V0.182 Zr0.182,BCC+Laves+Sec.,CAST,other,NaN,NaN,7.3,NaN,C,25.0,1540.0,1643.0,16.8,NaN,NaN,94.0,NaN,NaN,NaN,10.4028/www.scientific.net/MSF.849.76,2016.0,Microstructure and Mechanical Properties of a ...,10.4028.www.scientific.net.MSF.849.76,Hf0.182Nb0.182Si0.091Ti0.182V0.182Zr0.182,HfNbSi0.5TiVZr,1540.0,0.0,7.75,464.0,293.0,0.0,0.00,0.0,10.4028.www.scientific.net.MSF.849.76,Hf0.182Nb0.182Si0.091Ti0.182V0.182Zr0.182
1542,1,771.0,427.0,0.50,10.4028.www.scientific.net.MSF.849.76,78.0,Hf0.182 Nb0.182 

##### Looking at original rows

In [108]:
original_df = df_result[df_result['original_idx'].notnull()]

In [109]:
len(original_df[original_df['PROPERTY: YS (MPa)_original'].notnull()])

692

In [110]:
# And then we can evaluated p/r/f1 for individual columns
# Finding that scikit-learn is broken for some reason
# from sklearn.metrics import accuracy_score
# classification_report(original_df['PROPERTY: YS (MPa)_original'], original_df['PROPERTY: YS (MPa)_extracted'])

##### Comparing match rows for a particular paper

In [111]:
nonmatch_df = df_result[df_result['original_idx'].isnull() | df_result['extracted_idx'].isnull()]
print(nonmatch_df.shape)


(1363, 41)


In [112]:
match_df = df_result[df_result['original_idx'].notnull() & df_result['extracted_idx'].notnull()]
print(match_df.shape)

(192, 41)


In [113]:
match_df.to_csv('matched_langchain_zeroshot.csv', encoding='utf8')

In [114]:
original_df = df_result[df_result['original_idx'].notnull()]
print(original_df.shape)

(818, 41)


In [115]:
groups = df_result.groupby('match_group').indices
groups

{'10.1002.adem.200300567': array([0, 1, 2, 3, 4, 5, 6, 7]),
 '10.1002.adem.202100765': array([ 8,  9, 10, 11, 12]),
 '10.1002.adma.201701678': array([13, 14, 15, 16, 17, 18, 19, 20, 21, 22]),
 '10.1007.s00339-019-2484-1': array([23, 24, 25, 26, 27, 28, 29, 30]),
 '10.1007.s10853-012-6260-2': array([31, 32, 33, 34, 35, 36, 37, 38, 39]),
 '10.1007.s11661-018-4646-8': array([40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56,
        57, 58, 59, 60, 61, 62, 63, 64, 65, 66]),
 '10.1007.s11665-017-2799-z': array([67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82]),
 '10.1007.s11837-012-0366-5': array([83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96]),
 '10.1007.s11837-019-03432-9': array([ 97,  98,  99, 100, 101, 102, 103, 104]),
 '10.1007.s11837-020-04557-y': array([105, 106, 107]),
 '10.1007.s12598-019-01310-6': array([108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119]),
 '10.1007.s40195-019-00921-3': array([120, 121, 122, 123, 124, 125, 126, 1

In [116]:
#Taking a look at the non-matched original versus extracted rows for an individual paper
doi = list(groups.keys())[2]

print('Matched rows:')
display(match_df[match_df['match_group']==doi][['normalized_formula_original',
                                                     'normalized_formula_extracted',
                                                     'FORMULA_original',
                                                     'FORMULA_extracted',
                                                     'PROPERTY: YS (MPa)_original',
                                                     'PROPERTY: YS (MPa)_extracted',
                                                     'match_group']])

print('Non-matched rows:')
display(nonmatch_df[nonmatch_df['match_group']==doi][['original_idx', 'extracted_idx',
                                                      'normalized_formula_original',
                                                     'normalized_formula_extracted',
                                                     'FORMULA_original',
                                                     'FORMULA_extracted',
                                                     'PROPERTY: YS (MPa)_original',
                                                     'PROPERTY: YS (MPa)_extracted',
                                                     'match_group']])

Matched rows:


,normalized_formula_original,normalized_formula_extracted,FORMULA_original,FORMULA_extracted,PROPERTY: YS (MPa)_original,PROPERTY: YS (MPa)_extracted,match_group


Non-matched rows:


,original_idx,extracted_idx,normalized_formula_original,normalized_formula_extracted,FORMULA_original,FORMULA_extracted,PROPERTY: YS (MPa)_original,PROPERTY: YS (MPa)_extracted,match_group
13,627.0,NaN,Hf0.278Ta0.167Ti0.278Zr0.278,NaN,Hf0.278 Ta0.167 Ti0.278 Zr0.278,NaN,750.0,NaN,10.1002.adma.201701678
14,628.0,NaN,Hf0.286Ta0.143Ti0.286Zr0.286,NaN,Hf0.286 Ta0.143 Ti0.286 Zr0.286,NaN,687.0,NaN,10.1002.adma.201701678
15,622.0,NaN,Hf0.25Ta0.25Ti0.25Zr0.25,NaN,Hf0.25 Ta0.25 Ti0.25 Zr0.25,NaN,1356.0,NaN,10.1002.adma.201701678
16,631.0,NaN,Hf0.294Ta0.118Ti0.294Zr0.294,NaN,Hf0.294 Ta0.118 Ti0.294 Zr0.294,NaN,354.0,NaN,10.1002.adma.201701678
17,NaN,404.0,NaN,Hf0.25Ta0.25Ti0.25Zr0.25,NaN,TaHfZrTi,NaN,0.0,10.1002.adma.201701678
18,NaN,405.0,NaN,Hf0.278Ta0.167Ti0.278Zr0.278,NaN,Ta0.6HfZrTi,NaN,0.0,10.1002.adma.201701678
19,NaN,406.0,NaN,Hf0.286Ta0.143Ti0.286Zr0.286,NaN,Ta0.5HfZrTi,NaN,0.0,10.1002.adma.201701678
20,NaN,407.0,NaN,Hf0.294Ta0.118Ti0.294Zr0.294,NaN,Ta0.4HfZrTi,NaN,0.0,10.1002.adma.201701678
21,NaN,408.0,NaN,,NaN,Ta0.4,NaN,0.0,10.1002.adma.201701678
22,NaN,409.0,NaN,None,NaN,TaxHfZrTi,NaN,0.0,10.1002.adma.201701678


In [117]:
extraction_df[extraction_df['doi'] ==doi]

,formula,yield_strength,grain_size,experimental_density,hardness,test_temperature,ultimate_tensile_strength,elongation,elongation_plastic,doi,normalized_formula
404,TaHfZrTi,0.0,100.0,0.0,0.0,0.0,1500.0,4.0,0.0,10.1002.adma.201701678,Hf0.25Ta0.25Ti0.25Zr0.25
405,Ta0.6HfZrTi,0.0,0.0,0.0,0.0,0.0,1100.0,20.0,0.0,10.1002.adma.201701678,Hf0.278Ta0.167Ti0.278Zr0.278
406,Ta0.5HfZrTi,0.0,0.0,0.0,0.0,0.0,1100.0,27.0,0.0,10.1002.adma.201701678,Hf0.286Ta0.143Ti0.286Zr0.286
407,Ta0.4HfZrTi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.1002.adma.201701678,Hf0.294Ta0.118Ti0.294Zr0.294
408,Ta0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.1002.adma.201701678,
409,TaxHfZrTi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.1002.adma.201701678,None


In [118]:
df_result[df_result['match_group']==doi][['original_idx', 'extracted_idx',
                                          'normalized_formula_original',
                                                     'normalized_formula_extracted',
                                                     'FORMULA_original',
                                                     'FORMULA_extracted',
                                                     'PROPERTY: YS (MPa)_original',
                                                     'PROPERTY: YS (MPa)_extracted',
                                                     'match_group']]

,original_idx,extracted_idx,normalized_formula_original,normalized_formula_extracted,FORMULA_original,FORMULA_extracted,PROPERTY: YS (MPa)_original,PROPERTY: YS (MPa)_extracted,match_group
13,627.0,NaN,Hf0.278Ta0.167Ti0.278Zr0.278,NaN,Hf0.278 Ta0.167 Ti0.278 Zr0.278,NaN,750.0,NaN,10.1002.adma.201701678
14,628.0,NaN,Hf0.286Ta0.143Ti0.286Zr0.286,NaN,Hf0.286 Ta0.143 Ti0.286 Zr0.286,NaN,687.0,NaN,10.1002.adma.201701678
15,622.0,NaN,Hf0.25Ta0.25Ti0.25Zr0.25,NaN,Hf0.25 Ta0.25 Ti0.25 Zr0.25,NaN,1356.0,NaN,10.1002.adma.201701678
16,631.0,NaN,Hf0.294Ta0.118Ti0.294Zr0.294,NaN,Hf0.294 Ta0.118 Ti0.294 Zr0.294,NaN,354.0,NaN,10.1002.adma.201701678
17,NaN,404.0,NaN,Hf0.25Ta0.25Ti0.25Zr0.25,NaN,TaHfZrTi,NaN,0.0,10.1002.adma.201701678
18,NaN,405.0,NaN,Hf0.278Ta0.167Ti0.278Zr0.278,NaN,Ta0.6HfZrTi,NaN,0.0,10.1002.adma.201701678
19,NaN,406.0,NaN,Hf0.286Ta0.143Ti0.286Zr0.286,NaN,Ta0.5HfZrTi,NaN,0.0,10.1002.adma.201701678
20,NaN,407.0,NaN,Hf0.294Ta0.118Ti0.294Zr0.294,NaN,Ta0.4HfZrTi,NaN,0.0,10.1002.adma.201701678
21,NaN,408.0,NaN,,NaN,Ta0.4,NaN,0.0,10.1002.adma.201701678
22,NaN,409.0,NaN,None,NaN,TaxHfZrTi,NaN,0.0,10.1002.adma.201701678


In [119]:
filtered_paper_df[filtered_paper_df['processed_doi'] ==doi]

,IDENTIFIER: Reference ID,FORMULA,PROPERTY: Microstructure,PROPERTY: Processing method,PROPERTY: BCC/FCC/other,PROPERTY: grain size ($\mu$m),PROPERTY: Exp. Density (g/cm$^3$),PROPERTY: Computed Density (g/cm$^3$),PROPERTY: HV,PROPERTY: Type of test,PROPERTY: Test temperature ($^\circ$C),PROPERTY: YS (MPa),PROPERTY: UTS (MPa),PROPERTY: Elongation (%),PROPERTY: Elongation plastic (%),PROPERTY: Exp. Young modulus (GPa),PROPERTY: Computed Young modulus (GPa),PROPERTY: O content (wppm),PROPERTY: N content (wppm),PROPERTY: C content (wppm),REFERENCE: doi,REFERENCE: year,REFERENCE: title,processed_doi,normalized_formula
622,88,Hf0.25 Ta0.25 Ti0.25 Zr0.25,BCC,CAST,BCC,NaN,NaN,10.2,NaN,T,25.0,1356.0,1452.0,4.0,NaN,NaN,112.0,NaN,NaN,NaN,10.1002/adma.201701678,2017.0,Phase-Transformation Ductilization of Brittle ...,10.1002.adma.201701678,Hf0.25Ta0.25Ti0.25Zr0.25
627,88,Hf0.278 Ta0.167 Ti0.278 Zr0.278,BCC+HCP,CAST,other,NaN,NaN,9.5,NaN,T,25.0,750.0,1110.0,NaN,NaN,NaN,104.0,NaN,NaN,NaN,10.1002/adma.201701678,2017.0,Phase-Transformation Ductilization of Brittle ...,10.1002.adma.201701678,Hf0.278Ta0.167Ti0.278Zr0.278
628,88,Hf0.286 Ta0.143 Ti0.286 Zr0.286,BCC+HCP,CAST,other,NaN,NaN,9.3,NaN,T,25.0,687.0,1119.0,29.9,NaN,NaN,101.0,NaN,NaN,NaN,10.1002/adma.201701678,2017.0,Phase-Transformation Ductilization of Brittle ...,10.1002.adma.201701678,Hf0.286Ta0.143Ti0.286Zr0.286
631,88,Hf0.294 Ta0.118 Ti0.294 Zr0.294,BCC+HCP,CAST,other,NaN,NaN,9.1,NaN,T,25.0,354.0,1100.0,30.6,NaN,NaN,99.0,NaN,NaN,NaN,10.1002/adma.201701678,2017.0,Phase-Transformation Ductilization of Brittle ...,10.1002.adma.201701678,Hf0.294Ta0.118Ti0.294Zr0.294
